Import useful modules 

In [1]:
import sys
import os
### discomment above two line ones to add the feelpp path with nirb module in the python environement
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
# print(sys.path)
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
import time
import json


Choose config file

In [2]:
config_file = os.path.join("/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg")
N = None # number of initiale snapshot
greedy_tol = 1.e-5 # convergence criteria of Greedy method
Nmax_greedy = 100
outdir = None

Set feelpp environment 

In [3]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType'][0]
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'][0])
config_nirb = feelpp.readJson(nirb_file)['nirb']

[ Starting Feel++ ] application feelpp-mor-nirb version 0.1 date 2023-Jan-23
 . feelpp-mor-nirb files are stored in /home/saigre/Documents/DELETE_ME/nirb/heat/np_1
 .. logfiles :/home/saigre/Documents/DELETE_ME/nirb/heat/np_1/logs
Reading /home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg...


Get number of initial snapshots. By defaults, it will take the value filled in the model path. 

In [4]:
nbSnap=N
if nbSnap==None:
    nbSnap = config_nirb['nbSnapshots']

Set the output path 

In [5]:
# doGreedy = config_nirb['greedy-generation']
doGreedy = False
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class : either from scratch or from a ToolboxModel already initialized

In [6]:
if False:
    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.initModel()

else:
    tb = ToolboxModel(**config_nirb)
    tb.initModel()

    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.setModel(tb)

[NIRB] ToolboxModel created, but the objects have not yet been initialized. Please call initModel() or setModel() to initialize the objects.
[loadMesh] Loading mesh in format geo+msh: "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo"
[loadMesh] Use default geo desc: /home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo 0.01 
heat(2,1)
[NIRB] Initialization done
[NIRB] Number of nodes on the fine mesh : 11860
[NIRB] ToolboxModel created, but the objects have not yet been initialized. Please call initModel() or setModel() to initialize the objects.
[NIRB Offline] Initialization done
[NIRB] Initialization done
[NIRB] Number of nodes on the fine mesh : 11860
heat(2,1)
[NIRB] Number of nodes on the coarse mesh : 156
[CRBmodelProperties] Loading Model Properties : "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.json"
[loadMesh] Loading mesh in format geo+msh: "/

In [7]:
nirb_off.generateOperators(coarse=True)

Get nirb reduced basis functions 

In [8]:
Xi_train=None

if doGreedy:
    _, Xi_train, _ = nirb_off.initProblemGreedy(500, greedy_tol, Nmax=Nmax_greedy, Xi_train=Xi_train, computeCoarse=True, samplingMode="log-random")
else:
    Xi_train = nirb_off.initProblem(nbSnap)
nirb_off.generateReducedBasis(regulParam=1.e-10)

print("Is L2 orthonormalized ?", nirb_off.checkL2Orthonormalized())


Running simulation with mu = [1.16e-01,2.49e+00,7.60e+00,4.23e+00]
Running simulation with mu = [1.00e+00,3.21e+00,5.54e+00,5.09e+00]
Running simulation with mu = [9.94e-01,7.52e+00,4.97e+00,6.82e+00]
Running simulation with mu = [9.89e-01,6.65e+00,6.66e-01,3.70e+00]
Running simulation with mu = [6.00e-01,7.78e-01,6.29e+00,3.71e+00]
Running simulation with mu = [2.24e-01,3.85e-01,5.62e-01,3.54e-01]
Running simulation with mu = [3.04e-01,5.87e+00,2.59e-01,2.52e+00]
Running simulation with mu = [3.19e-01,9.60e+00,5.44e-01,5.17e+00]
Running simulation with mu = [4.16e-01,9.51e-01,8.30e+00,2.37e+00]
Running simulation with mu = [5.92e-01,6.73e+00,5.05e+00,3.96e+00]
[NIRB] Number of snapshot computed : 10
[NIRB] Gram-Schmidt L2 orthonormalization done after 0 step
[NIRB] Number of modes : 10
Is L2 orthonormalized ? True


Save datas. 

If `outdir`is given, the results will be saved on this path. otherwise they are saved in : "HOME/feelppdb/nirb/{ToolboxType}/np_{k}/{RESPATH}" where `k` is the number of process. 

In [9]:
nirb_off.saveData(RESPATH, force=True)

[NIRB] Data saved in /home/saigre/Documents/DELETE_ME/nirb/heat/np_1/results/noRect/noGreedy


Get infos 

In [10]:
info = nirb_off.getInformations()

if feelpp.Environment.isMasterRank():
    print(json.dumps(info, sort_keys=True, indent=4))
    print(f"[NIRB] Offline part Done !")

{
    "H": 0.1,
    "NH": 156,
    "Ndofs": 11860,
    "Nh": 11860,
    "dimension": 2,
    "h": 0.010000000000000002,
    "order": 1,
    "outdir": "/home/saigre/Documents/DELETE_ME/nirb/heat/np_1/results/noRect/noGreedy",
    "toolboxType": "heat"
}
[NIRB] Offline part Done !
